## AI Midterm Project
Fiorella Averina Gunawan <br>
19/443579/TK/48775 <br>
<br>
Decision Tree Algorithm (from Scratch) to Predict Whether A User Will Buy A Car or Not<br>
Reference: https://github.com/SebastianMantey/Decision-Tree-from-Scratch/blob/master/notebooks/Video%2008%20-%20Categorical%20Features.ipynb, Python NumPy and Pandas documentation <br>
With Modification (in this project, Gini Impurity Index will be used instead of Entropy) <br>
Dataset is taken from Kaggle <br>
Link: https://www.kaggle.com/datasets/gabrielsantello/cars-purchase-decision-dataset/code

## Importing Libraries

In [9]:
import numpy as np
import pandas as pd
import random
from pprint import pprint

## Importing Data and Data Preprocessing

In [10]:
url = 'https://drive.google.com/file/d/1X2OCBiCBiwBBkflIr8FsFYafP2mtA9gA/view?usp=sharing' #Define the URL link
file_id=url.split('/')[-2] #Extracting the file ID
download_url='https://drive.google.com/uc?id=' + file_id #Creating the download URL
df = pd.read_csv(download_url, sep=',') #Reading the CSV from the download URL, using ',' as the separator
df.head() #Printing some values of the dataframe

,User ID,Gender,Age,AnnualSalary,Purchased
0,385,Male,35,20000,0
1,681,Male,40,43500,0
2,353,Male,49,74000,0
3,895,Male,40,107500,1
4,661,Male,25,79000,0


In [11]:
df = df.drop("User ID", axis=1) #Removing the "User ID" column
#It is removed since it only functions as an identifier for every customer
#We want to predict whether the customer would buy a car or not, by using these attributes only: gender, age, annual salary
df = df.rename(columns={"Purchased": "label"}) #Renaming the last column into "label", so that it would be easily processed
#(The functions are written using the "label" parameters)
df.head() #Rechecking the data

,Gender,Age,AnnualSalary,label
0,Male,35,20000,0
1,Male,40,43500,0
2,Male,49,74000,0
3,Male,40,107500,1
4,Male,25,79000,0


In [12]:
df.info() #Checking the data info, to see whether there exists some NaN values in our dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Gender        1000 non-null   object
 1   Age           1000 non-null   int64 
 2   AnnualSalary  1000 non-null   int64 
 3   label         1000 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 31.4+ KB


## Splitting Data into Training and Testing

In [13]:
#Input: dataframe df, test_size -> mau berapa persen, mau berapa data
def train_test_split (df, test_size, seed): #Test size must be integer
    #If test size is a percentage (float), then test_size = the percentage times the dataframe
    if isinstance(test_size, float): #Check whether the desired test size is a float
        test_size = round(test_size * len(df)) #Calculating the test size if the test size is a float
        #It is rounded, since test size must be integer

    indices = df.index.tolist() #Mengubah indeks data menjadi list untuk di-sampling
    test_indices = random.sample(population=indices, k=test_size) #Indeks data untuk dijadikan testing data

    test_df = df.loc[test_indices] #Mengakses data dengan indeks test_indices
    #Data tersebut menjadi testing dataframe
    train_df = df.drop(test_indices) #Mengakses data yang indeksnya selain test_indices
    #Data tersebut menjadi training dataframe
    random.seed(seed) #Menentukan set random ke berapa, agar data random tidak berubah setiap perintah dijalankan
    return train_df, test_df


## Helper Functions

In [14]:
train_df, test_df = train_test_split(df, test_size = 0.2, seed = 0)

In [15]:
#Converting pandas dataframe "df" to numpy 2D array "data"
data = train_df.values

In [16]:
#Check data purity. Purity = 1 if there exists only one unique value in the label
def check_purity(data):
    label_column = data[:,-1] #Mengambil isi dari kolom terakhir (label)
    unique_classes = np.unique(label_column) #Counting the unique values

    if len(unique_classes) == 1:
        return True
    else:
        return False

In [17]:
#Classify data
#Data will be classified to a unique class that has most counts
def classify_data(data):
    label_column = data[:,-1]
    unique_classes, counts_unique_classes = np.unique (label_column, return_counts=True)

    index = counts_unique_classes.argmax() #Returns the index of the unique class that has the most data
    classification = unique_classes[index] #Classify it as a unique class with the index of "index"

    return classification

In [18]:
#Potential Splits Function (to get all the possibilities of data splitting)
def get_potential_splits(data):
    
    potential_splits = {} #Defining potential_splits as a dictionary
    _, n_columns = data.shape #Accessing the column dimension of data
    for column_index in range(n_columns - 1): #Every column has its potential split, except the last column
        #which is the "label" column
        values = data[:, column_index] #Accesing the value of the data at a column
        unique_values = np.unique(values) #Getting the unique values of the data at the respective column
        
        type_of_feature = FEATURE_TYPES[column_index] #Determining types of the feature
        #Possibilities: continuous (e.g. age which is numerical), and categorical (e.g. gender which contains strings data type)

        #For continuous features:
        if type_of_feature == "Continuous":
            potential_splits[column_index] = [] #Initiating an array that contains values that could be used for splitting the data
            for index in range(len(unique_values)): #Looping along the number of unique values to find potential splits
                if index != 0:
                    current_value = unique_values[index] #Storing the current value in the index n of unique values array
                    previous_value = unique_values[index - 1] #Storing the previous value in the index of n-1 of unique values array
                    potential_split = (current_value + previous_value) / 2 #Calculating the potential split value

                    potential_splits[column_index].append(potential_split) #Adding the potntial split value to the array
        
        #For categorical features with more than 1 unique value:
        elif len(unique_values) > 1:
            potential_splits[column_index] = unique_values #The splitting value is itself (e.g. Male or Female for gender attribute)
    
    return potential_splits

In [19]:
#Splitting the Data
def split_data(data, split_column, split_value):
    split_column_values = data[:, split_column] #Getting the value that will be used as reference to split the data
    type_of_feature = FEATURE_TYPES[split_column] #Getting its feature types
    #Classify the data
    if type_of_feature == "Continuous":
        data_yes = data[split_column_values <= split_value]
        data_no = data[split_column_values > split_value]

    else: #For categorical data, check its equality
        data_yes = data[split_column_values == split_value]
        data_no = data[split_column_values != split_value]
        
    return data_yes, data_no

In [20]:
# Calculate Gini Impurity
def calculate_gini(data):
    label_column = data[:,-1] #Access the "label" column
    _, counts = np.unique(label_column, return_counts=True) #Accessing how many times the unique value appears

    probabilities = counts/counts.sum() #Returning the array containing the probabilites of each unique value
    gini = 1 - (sum(np.square(probabilities))) #Calculating gini impurity

    return gini

In [21]:
#Calculating total gini (weighted average)
def calculate_total_gini (data_yes, data_no):
    p_data_yes = len(data_yes)/len(data) #Probability of "yes" data
    p_data_no = len(data_no)/len(data) #Probability of "no" data

    #Weighted Average
    total_gini = (p_data_yes*calculate_gini(data_yes) + p_data_no*calculate_gini(data_no))

    return total_gini

In [22]:
#Determining best split
def determine_best_split(data, potential_splits):
    overall_gini = 9999 #Very large, tidak mungkin mencapai nilai ini
    for column_index in potential_splits: #Looping along each column for potential splits
        for value in potential_splits[column_index]: #Looping along each value in every column for potential splits
            data_yes, data_no = split_data(data, split_column=column_index, split_value=value) #Splitting the data
            current_total_gini = calculate_total_gini(data_yes, data_no) #Calculating current split's total gini
            
            if current_total_gini <= overall_gini: #Comparing the total gini
                overall_gini = current_total_gini #Substituing overall gini to current total gini
                best_split_column = column_index #Best split column is the respective column index
                best_split_value = value #Best split value is the respective value in column index

    return best_split_column, best_split_value

## Decision Tree Algorithm

In [23]:
#We want to creat a sub tree which is a dictionary, that contains the yes answer and the no answer
#sub_tree = {"question": ["yes_answer", "no_answer"]}

In [24]:
def determine_type_of_feature(df): #Categorical or continuous?

    feature_types = [] #Initializing an array to store the feature type
    threshold = 5 #Sebagai limit yang menandakan feature termasuk categorical atau continuous. 
    #Data yang continuous tentunya memiliki sangat banyak unique value
    #Threshold berfungsi sebagai batas unique value yang bisa dimiliki feature agar dikategorikan sebagai "categorical"
    
    for feature in df.columns: #Looping along each features in columns
        if feature != "label": #If the feature is not the label,
            unique_values = df[feature].unique() #Getting the array that contains unique values for each feature
            example_value = unique_values[0] #Setting the example value as the first entry of unique_values array

            #Check if the example value is categorical. Checking if its data type is string, or it has unique values less than the threshold
            if (isinstance(example_value, str)) or (len(unique_values) <= threshold):
                feature_types.append("Categorical")
            else: #Categorize the feature as a continuous one
                feature_types.append("Continuous")
    return feature_types

In [25]:
#Main algorithm
def decision_tree_algorithm(df, i=0, min_samples=5, max_depth=3):
    
    if i == 0: #First loop:
        global COLUMN_HEADERS, FEATURE_TYPES #Initialize global variables
        COLUMN_HEADERS = df.columns #Defining the column headers variable
        FEATURE_TYPES = determine_type_of_feature(df) #Defining the feature types
        data = df.values #Taking the data in the form of NumPy 2D array (during the first execution of the loop)
    else:
        data = df           
    
    
    #Non-recursive
    #Happens when the data is pure, or the number of the data points is less than the minimum samples,
    #or the counter "i" has reached is maximum value (max_depth)
    #What happens after this: classify the data
    if (check_purity(data)) or (len(data) < min_samples) or (i == max_depth):
        classification = classify_data(data) #Classifying the data
        
        return classification

    
    #Recursive, happens when the above conditions are not meet
    else:    
        i = i + 1 #Incrementing the counter

        potential_splits = get_potential_splits(data) #Getting potential splits
        split_column, split_value = determine_best_split(data, potential_splits) #Getting split column and split value
        data_yes, data_no = split_data(data, split_column, split_value) #Splitting the data
        
        #Checking in case there is an empty data
        if len(data_yes) == 0 or len(data_no) == 0:
            classification = classify_data(data) #Classify
            return classification
        
        feature_name = COLUMN_HEADERS[split_column] #Getting feature name
        type_of_feature = FEATURE_TYPES[split_column] #Getting feature type

        #For continuous data
        #Question: "Is ... (feature name) <= ... (split value)?"
        if type_of_feature == "Continuous":
            question = "{} <= {}".format(feature_name, split_value)
            
        #For categorical data
        #Question: "Is ... (feature name) = ... (split value)?"
        else:
            question = "{} = {}".format(feature_name, split_value)
        
        #Initializing subtree
        #The tree will consists of some subtrees
        sub_tree = {question: []} #Storing the questions and their answers in the sub_tree dictionary
        
        #Answers
        yes_answer = decision_tree_algorithm(data_yes, i, min_samples, max_depth) #Run the algorithm for the subtree for yes answers
        no_answer = decision_tree_algorithm(data_no, i, min_samples, max_depth) #Run the algorithm for the subtree for no answers
        
        #Same answer case (e.g. the classification of the "yes" and "no" sides are the same):
        if yes_answer == no_answer:
            sub_tree = yes_answer #Choose either value (it is the same)

        #Different answer case:
        else:
            sub_tree[question].append(yes_answer) #Generating the subtree for yes answer
            sub_tree[question].append(no_answer) #Generating the subtree for no answer
        
        return sub_tree

## Classification

In [26]:
def classify(example, tree):

    #Getting the question
    question = list(tree.keys())[0] #Converting the keys to a list, accessing the first element
    feature_name, comparison_operator, value = question.split(" ") #Splitting the question, into 3 parts
    #Which are feature names, comparison operators, and values

    #For continuous features
    if comparison_operator == "<=":
        if example[feature_name] <= float(value): #Question
            answer = tree[question][0] #Pick the 'yes answer' (the first element in the value list)
        else:
            answer = tree[question][1] #Pick the 'no answer' (the second element in the value list)
    
    #For categorical features
    else:
        if str(example[feature_name]) == value: #Question
            answer = tree[question][0] #Pick the 'yes answer'
        else:
            answer = tree[question][1] #Pick the 'no answer'

    #Recursive
    if isinstance(answer, dict): #If the answer is a dictionary (subtree)
        residual_tree = answer #Answer is the subtree (residual)
        return classify(example, residual_tree) #Run classify function for the subtree
    
    
    #Non-recursive
    else:
        return answer #Return the answer (classification)


## Calculating Accuracy

In [27]:
def calculate_accuracy(df, tree):

    #Creating "Classification" column in dataframe
    df["Classification"] = df.apply(classify, axis = 1, args = (tree,)) #tree as tuple of 1 column 
    #That was for applying classify function along every row
    
    #Creating "Classification True/False?" that consists of boolean values
    df["Correctness"] = df["Classification"] == df["label"]
    #Calculating the accuracy
    #Pandas will convert "True" to 1 and "False" to 0 and calculate the mean
    #The higher the mean is, the more "True" answers it indicates, the more accurate the decision tree model is
    accuracy = df["Correctness"].mean()
    
    return accuracy

## Using our Car Purchase Decision Dataset

In [28]:
train_df, test_df = train_test_split(df, 0.2, 0)
tree = decision_tree_algorithm(train_df)
accuracy = calculate_accuracy(test_df, tree)

pprint (tree)

{'Age <= 45.5': [{'AnnualSalary <= 92000.0': [0, 1]}, 1]}


In [29]:
print (accuracy)

0.935


In [30]:
test_df

,Gender,Age,AnnualSalary,label,Classification,Correctness
864,Male,43,150500,1,1,True
394,Male,29,60500,0,0,True
776,Female,36,58500,0,0,True
911,Female,32,117000,1,1,True
430,Male,21,88000,0,0,True
...,...,...,...,...,...,...
587,Male,18,82000,0,0,True
649,Male,40,54500,0,0,True
547,Female,49,135500,1,1,True
965,Male,52,150000,1,1,True
